In [1]:
import cv2
import os
from datetime import datetime
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from ultralytics import YOLO
import numpy as np

img_width = 256
img_height = 65
max_length = 8

# Load OCR model
model_path = "C:/Users/sepan/Desktop/Persian-Car-License-Plate-Recognition-for-Company-Camera-master/Prediction of license plate characters/OCR.keras"
ocr_model = keras.models.load_model(model_path)

# Load YOLO model
yolo_model = YOLO('C:/Users/sepan/Desktop/Persian-Car-License-Plate-Recognition-for-Company-Camera-master/Detecting the position of the license plate/best.pt')

characters = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'H', 'J', 'L', 'M', 'N', 'P', 'Q', 'S', 'T', 'V', 'W', 'X', 'Y', 'Z', '»']
char_to_num = layers.StringLookup(vocabulary=characters, mask_token=None)
num_to_char = layers.StringLookup(vocabulary=char_to_num.get_vocabulary(), mask_token=None, invert=True)

def capture_image(frame):
    now = datetime.now()
    filename = now.strftime("%Y%m%d_%H%M%S") + ".jpg"
    save_path = r"C:\Users\sepan\Desktop\test"
    os.makedirs(save_path, exist_ok=True)
    full_path = os.path.join(save_path, filename)
    cv2.imwrite(full_path, frame)
    print(f"Image saved at {full_path}")
    return full_path

def preprocess_image(image):
    if isinstance(image, str):
        img = tf.io.read_file(image)
        img = tf.io.decode_jpeg(img, channels=1)
    else:
        img = tf.image.rgb_to_grayscale(image)
    
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, [img_height, img_width])
    img = tf.transpose(img, perm=[1, 0, 2])
    img = tf.expand_dims(img, axis=0)
    return img

def predict_label(img):
    pred = ocr_model.predict(img)
    pred = pred.argmax(axis=-1)
    predicted_label = tf.strings.reduce_join(num_to_char(pred)).numpy().decode("utf-8")
    return predicted_label

def process_image(image_path):
    img = Image.open(image_path)
    img_with_boxes = img.copy()
    draw = ImageDraw.Draw(img_with_boxes)

    results = yolo_model(img)
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()
        labels = result.boxes.cls.cpu().numpy()

        for i, box in enumerate(boxes):
            if labels[i] == 0:
                x1, y1, x2, y2 = map(int, box)
                draw.rectangle([x1, y1, x2, y2], outline="red", width=3)
                plate_img = img.crop((x1, y1, x2, y2))
                
                # Convert PIL Image to numpy array for OCR
                plate_img_np = np.array(plate_img.convert('L'))
                plate_img_np = np.expand_dims(plate_img_np, axis=-1)  # Add channel dimension
                plate_img_np = np.repeat(plate_img_np, 3, axis=-1)  # Repeat to create 3 channels
                plate_img_tf = preprocess_image(tf.convert_to_tensor(plate_img_np))
                predicted_label = predict_label(plate_img_tf)

                plt.figure(figsize=(12, 4))
                plt.subplot(1, 3, 1)
                plt.imshow(img_with_boxes)
                plt.title('Image with Bounding Boxes')
                plt.axis('off')

                plt.subplot(1, 3, 2)
                plt.imshow(plate_img, cmap='gray')
                plt.title('Detected License Plate')
                plt.axis('off')

                plt.subplot(1, 3, 3)
                plt.text(0.5, 0.5, f"Predicted: {predicted_label}", 
                         fontsize=20, ha='center', va='center')
                plt.axis('off')

                plt.tight_layout()
                plt.show()

def main():
    cap = cv2.VideoCapture(0)
    print("Program ready. Press Space to capture an image. Press 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error reading from webcam.")
            break

        cv2.imshow('Webcam', frame)

        key = cv2.waitKey(1)
        if key == ord(' '):
            image_path = capture_image(frame)
            process_image(image_path)
        elif key == ord('q'):
            print("Program terminated.")
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

ValueError: File not found: filepath=C:/Users/sepan/Desktop/Persian-Car-License-Plate-Recognition-for-Company-Camera-master/Prediction of license plate characters/OCR.keras. Please ensure the file is an accessible `.keras` zip file.